## **Cuaderno Educacion**

In [58]:
import pandas as pd
import numpy as np
import os
import sys

In [ ]:
municipios_cluster = pd.read_csv(r"external_data/municipios_con_cluster.csv", sep=",")

### **profesores/alumnos_pub_100**

In [60]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
ESTUDIANTES_CSV = r"data/educacion/estudiantes_publicos.csv"   # sep=";"
PROFESORES_CSV  = r"data/educacion/profesores_publico.csv"     # sep=";"

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, '13,00' -> 13.0"""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 1) CARGA: ALUMNOS PÚBLICOS (ancho)
# ============================================================
alumnos = pd.read_csv(ESTUDIANTES_CSV, sep=";", dtype=str)
alumnos["Nombre"] = alumnos["Nombre"].apply(normalize_nombre)
alumnos["key"] = alumnos["Nombre"].apply(canon_key)

# Columnas año
year_cols_alum = [c for c in alumnos.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_alum:
    alumnos[c] = alumnos[c].apply(parse_es_number)

alumnos_publicos = alumnos[["key", "Nombre"] + year_cols_alum].copy()

# ============================================================
# 2) CARGA: PROFESORES PÚBLICOS (ancho)
# ============================================================
prof = pd.read_csv(PROFESORES_CSV, sep=";", dtype=str)
prof["Nombre"] = prof["Nombre"].apply(normalize_nombre)
prof["key"] = prof["Nombre"].apply(canon_key)

year_cols_prof = [c for c in prof.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_prof:
    prof[c] = prof[c].apply(parse_es_number)

profesores_publicos = prof[["key", "Nombre"] + year_cols_prof].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
#     Debe tener columnas: Nombre, Cluster
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección alumnos vs profesores) >= 2022
# ============================================================
years_alum = {int(c) for c in year_cols_alum}
years_prof = {int(c) for c in year_cols_prof}

years_to_process = sorted(
    y for y in (years_alum.intersection(years_prof))
    if y >= 2022
)

print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    # Selección por año
    df_prof = profesores_publicos[["key", "Nombre", col]].rename(columns={col: "profesores_publicos"})
    df_alum = alumnos_publicos[["key", col]].rename(columns={col: "alumnos_publicos"})

    # Merge por key
    datos = pd.merge(df_prof, df_alum, on="key", how="inner")

    # Añadir cluster y (opcional) nombre del cluster como nombre final
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    # Numéricos
    datos["profesores_publicos"] = pd.to_numeric(datos["profesores_publicos"], errors="coerce")
    datos["alumnos_publicos"] = pd.to_numeric(datos["alumnos_publicos"], errors="coerce")

    # Profesores por 100 alumnos (evitar división por 0)
    datos["profesores_pub_100_alumnos_pub"] = (
        (datos["profesores_publicos"] / datos["alumnos_publicos"]) * 100
    ).replace([float("inf"), -float("inf")], 0).fillna(0)

    # type=1 => más es mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["profesores_pub_100_alumnos_pub"].mean()
        desviacion = grupo["profesores_pub_100_alumnos_pub"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    # Con/sin cluster (igual que antes)
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "profesores_pub_100_alumnos_pub"}
    )

    # Guardar
    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}profesores_pub_100_alumnos_pub.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2022, 2023, 2024]
Archivo del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/
Archivo del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\3638975320.py:164: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\3638975320.py:164: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **profesores/alumnos_priv_100**

In [61]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
ESTUDIANTES_CSV = r"data/educacion/estudiantes_privados.csv"   # sep=";"
PROFESORES_CSV  = r"data/educacion/profesores_privado.csv"     # sep=";"

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, '13,00' -> 13.0"""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 1) CARGA: ALUMNOS PÚBLICOS (ancho)
# ============================================================
alumnos = pd.read_csv(ESTUDIANTES_CSV, sep=";", dtype=str)
alumnos["Nombre"] = alumnos["Nombre"].apply(normalize_nombre)
alumnos["key"] = alumnos["Nombre"].apply(canon_key)

# Columnas año
year_cols_alum = [c for c in alumnos.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_alum:
    alumnos[c] = alumnos[c].apply(parse_es_number)

alumnos_privado = alumnos[["key", "Nombre"] + year_cols_alum].copy()

# ============================================================
# 2) CARGA: PROFESORES PÚBLICOS (ancho)
# ============================================================
prof = pd.read_csv(PROFESORES_CSV, sep=";", dtype=str)
prof["Nombre"] = prof["Nombre"].apply(normalize_nombre)
prof["key"] = prof["Nombre"].apply(canon_key)

year_cols_prof = [c for c in prof.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_prof:
    prof[c] = prof[c].apply(parse_es_number)

profesores_privados = prof[["key", "Nombre"] + year_cols_prof].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
#     Debe tener columnas: Nombre, Cluster
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección alumnos vs profesores) >= 2022
# ============================================================
years_alum = {int(c) for c in year_cols_alum}
years_prof = {int(c) for c in year_cols_prof}

years_to_process = sorted(
    y for y in (years_alum.intersection(years_prof))
    if y >= 2022
)

print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    # Selección por año
    df_prof = profesores_privados[["key", "Nombre", col]].rename(columns={col: "profesores_privados"})
    df_alum = alumnos_privado[["key", col]].rename(columns={col: "alumnos_privado"})

    # Merge por key
    datos = pd.merge(df_prof, df_alum, on="key", how="inner")

    # Añadir cluster y (opcional) nombre del cluster como nombre final
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    # Numéricos
    datos["profesores_privados"] = pd.to_numeric(datos["profesores_privados"], errors="coerce")
    datos["alumnos_privado"] = pd.to_numeric(datos["alumnos_privado"], errors="coerce")

    # Profesores por 100 alumnos (evitar división por 0)
    datos["profesores_priv_100_alumnos_priv"] = (
        (datos["profesores_privados"] / datos["alumnos_privado"]) * 100
    ).replace([float("inf"), -float("inf")], 0).fillna(0)

    # type=1 => más es mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["profesores_priv_100_alumnos_priv"].mean()
        desviacion = grupo["profesores_priv_100_alumnos_priv"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    # Con/sin cluster (igual que antes)
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "profesores_priv_100_alumnos_priv"}
    )

    # Guardar
    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}profesores_priv_100_alumnos_priv.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2022, 2023, 2024]
Archivo del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/
Archivo del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\2112352533.py:164: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\2112352533.py:164: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **centrosPublicosporAlumno_100**

In [62]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# CONFIG: rutas de entrada
# -----------------------------
ESTUDIANTES_CSV = r"data/educacion/estudiantes_publicos.csv"   # sep=";"
CENTROS_CSV  = r"data/educacion/centros_publico.csv"     # sep=";"

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    """Convierte '4.868' -> 4868, '26.226,00' -> 26226.0, '13,00' -> 13.0"""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """
    - Quita código INE al inicio: '28001 Acebeda, La' -> 'Acebeda, La'
    - Convierte 'X, La/El/Los/Las' -> 'X (La/El/Los/Las)'
    """
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE

    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar nombres aunque haya tildes, comas, espacios, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 1) CARGA: ALUMNOS PÚBLICOS (ancho)
# ============================================================
alumnos = pd.read_csv(ESTUDIANTES_CSV, sep=";", dtype=str)
alumnos["Nombre"] = alumnos["Nombre"].apply(normalize_nombre)
alumnos["key"] = alumnos["Nombre"].apply(canon_key)

# Columnas año
year_cols_alum = [c for c in alumnos.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_alum:
    alumnos[c] = alumnos[c].apply(parse_es_number)

alumnos_publicos = alumnos[["key", "Nombre"] + year_cols_alum].copy()

# ============================================================
# 2) CARGA: PROFESORES PÚBLICOS (ancho)
# ============================================================
centros = pd.read_csv(CENTROS_CSV, sep=";", dtype=str)
centros["Nombre"] = centros["Nombre"].apply(normalize_nombre)
centros["key"] = centros["Nombre"].apply(canon_key)

year_cols_prof = [c for c in centros.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_prof:
    centros[c] = centros[c].apply(parse_es_number)

centros_publicos = centros[["key", "Nombre"] + year_cols_prof].copy()

# ============================================================
# 3) PREPARAR CLUSTERS (municipios_cluster ya cargado)
#     Debe tener columnas: Nombre, Cluster
# ============================================================
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 4) AÑOS A PROCESAR (intersección alumnos vs centros ) >= 2022
# ============================================================
years_alum = {int(c) for c in year_cols_alum}
years_prof = {int(c) for c in year_cols_prof}

years_to_process = sorted(
    y for y in (years_alum.intersection(years_prof))
    if y >= 2022
)

print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 5) FUNCIÓN PRINCIPAL
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    # Selección por año
    df_prof = centros_publicos[["key", "Nombre", col]].rename(columns={col: "centros_publicos"})
    df_alum = alumnos_publicos[["key", col]].rename(columns={col: "alumnos_publicos"})

    # Merge por key
    datos = pd.merge(df_prof, df_alum, on="key", how="inner")

    # Añadir cluster y (opcional) nombre del cluster como nombre final
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    # Numéricos
    datos["centros_publicos"] = pd.to_numeric(datos["centros_publicos"], errors="coerce")
    datos["alumnos_publicos"] = pd.to_numeric(datos["alumnos_publicos"], errors="coerce")

    # Profesores por 100 alumnos (evitar división por 0)
    datos["centros_pub_100_alumnos_pub"] = (
        (datos["centros_publicos"] / datos["alumnos_publicos"]) * 100
    ).replace([float("inf"), -float("inf")], 0).fillna(0)

    # type=1 => más es mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["centros_pub_100_alumnos_pub"].mean()
        desviacion = grupo["centros_pub_100_alumnos_pub"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    # Con/sin cluster (igual que antes)
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "centros_pub_100_alumnos_pub"}
    )

    # Guardar
    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}centros_pub_100_alumnos_pub.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 6) PROCESAR TODOS LOS AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2022, 2023, 2024]
Archivo del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/
Archivo del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\1147860198.py:164: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\1147860198.py:164: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **centrosPrivadosporAlumno_100**

In [63]:
estudiantes_publicos = pd.read_csv(r"data/educacion/estudiantes_publicos.csv", sep=";")
estudiantes_privados = pd.read_csv(r"data/educacion/estudiantes_privados.csv", sep=";")
centros_privados = pd.read_csv("data/educacion/centros_privado.csv", sep=";", usecols=lambda col: not col.startswith("Unnamed"))

In [ ]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

# -----------------------------
# Helpers (igual que antes)
# -----------------------------
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 0) PREPARAR DATAFRAMES (estudiantes_privados, centros_privados, municipios_cluster)
#    (asumo que ya existen en memoria como DataFrames)
# ============================================================

# --- Estudiantes privados ---
estudiantes_privados = estudiantes_privados.copy()
estudiantes_privados["Nombre"] = estudiantes_privados["Nombre"].apply(normalize_nombre)
estudiantes_privados["key"] = estudiantes_privados["Nombre"].apply(canon_key)
year_cols_est = [c for c in estudiantes_privados.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_est:
    estudiantes_privados[c] = estudiantes_privados[c].apply(parse_es_number)

# --- Centros privados ---
centros_privados = centros_privados.copy()
centros_privados["Nombre"] = centros_privados["Nombre"].apply(normalize_nombre)
centros_privados["key"] = centros_privados["Nombre"].apply(canon_key)
year_cols_cent = [c for c in centros_privados.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_cent:
    centros_privados[c] = centros_privados[c].apply(parse_es_number)

# --- Clusters ---
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 1) AÑOS A PROCESAR: comunes y >= 2022
# ============================================================
years_est = {int(c) for c in year_cols_est}
years_cent = {int(c) for c in year_cols_cent}

years_to_process = sorted(y for y in (years_est.intersection(years_cent)) if y >= 2022)
print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 2) FUNCIÓN POR AÑO
# ============================================================
def procesar_anio(anio: int):
    print(f"\nProcesando año: {anio}")
    col = str(int(anio))

    alumnos = estudiantes_privados[["key", "Nombre", col]].rename(columns={col: "estudiantes_privados"})
    centros = centros_privados[["key", col]].rename(columns={col: "centros_privados"})

    # Outer merge para no perder municipios
    datos = pd.merge(alumnos, centros, on="key", how="outer")

    # ✅ Arreglo: solo unir Nombre_x/Nombre_y si existen (si no, no tocar)
    if "Nombre_x" in datos.columns and "Nombre_y" in datos.columns:
        datos["Nombre"] = datos["Nombre_x"].fillna(datos["Nombre_y"])
        datos = datos.drop(columns=["Nombre_x", "Nombre_y"], errors="ignore")

    datos["estudiantes_privados"] = pd.to_numeric(datos["estudiantes_privados"], errors="coerce").fillna(0)
    datos["centros_privados"] = pd.to_numeric(datos["centros_privados"], errors="coerce").fillna(0)

    datos["centros_priv_100_alumnos_priv"] = (datos["centros_privados"] / datos["estudiantes_privados"]) * 100
    datos["centros_priv_100_alumnos_priv"] = (
        datos["centros_priv_100_alumnos_priv"]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(0)
        .round(2)
    )

    datos["type"] = 1

    # Añadir cluster + nombre final desde cluster si existe
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"], errors="ignore")

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["centros_priv_100_alumnos_priv"].mean()
        desviacion = grupo["centros_priv_100_alumnos_priv"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)
        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "centros_priv_100_alumnos_priv"}
    )

    print(f"Municipios sin valor final (NaN): {resultado['centros_priv_100_alumnos_priv'].isna().sum()}")

    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)

    resultado.to_csv(f"{output_path}centros_priv_100_alumnos_priv.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")


# ============================================================
# 3) PROCESAR AÑOS (>=2022)
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2022, 2023, 2024]

📅 Procesando año: 2022
🎓 Alumnos no nulos: 179
🏫 Centros no nulos: 179
🔗 Municipios tras merge: 179
Municipios sin valor final (NaN): 0
Archivo del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/

📅 Procesando año: 2023
🎓 Alumnos no nulos: 179
🏫 Centros no nulos: 179
🔗 Municipios tras merge: 179
Municipios sin valor final (NaN): 0
Archivo del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/

📅 Procesando año: 2024
🎓 Alumnos no nulos: 179
🏫 Centros no nulos: 179
🔗 Municipios tras merge: 179
Municipios sin valor final (NaN): 0
Archivo del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\493252638.py:146: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\493252638.py:146: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_at

### **unidadEsc/100_alumnos_pub**

In [65]:
estudiantes_publicos = pd.read_csv("data/educacion/estudiantes_publicos.csv", sep=";")
unidades_escolares_publicas = pd.read_csv("data/educacion/unidades_esc_publicos.csv", sep=";")

In [66]:
import os
import re
import unicodedata
import pandas as pd

# -----------------------------
# Helpers (igual que antes)
# -----------------------------
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # sin tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 0) PREPARAR DATAFRAMES (estudiantes_publicos, unidades_escolares_publicas, municipios_cluster)
# ============================================================
# --- Estudiantes públicos ---
estudiantes_publicos = estudiantes_publicos.copy()
estudiantes_publicos["Nombre"] = estudiantes_publicos["Nombre"].apply(normalize_nombre)
estudiantes_publicos["key"] = estudiantes_publicos["Nombre"].apply(canon_key)

year_cols_est = [c for c in estudiantes_publicos.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_est:
    estudiantes_publicos[c] = estudiantes_publicos[c].apply(parse_es_number)

# --- Unidades escolares públicas ---
unidades_escolares_publicas = unidades_escolares_publicas.copy()
unidades_escolares_publicas["Nombre"] = unidades_escolares_publicas["Nombre"].apply(normalize_nombre)
unidades_escolares_publicas["key"] = unidades_escolares_publicas["Nombre"].apply(canon_key)

year_cols_uni = [c for c in unidades_escolares_publicas.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_uni:
    unidades_escolares_publicas[c] = unidades_escolares_publicas[c].apply(parse_es_number)

# --- Clusters ---
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 1) AÑOS A PROCESAR: comunes y >= 2022
# ============================================================
years_est = {int(c) for c in year_cols_est}
years_uni = {int(c) for c in year_cols_uni}

years_to_process = sorted(y for y in (years_est.intersection(years_uni)) if y >= 2022)
print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 2) FUNCIÓN POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    alumnos = estudiantes_publicos[["key", "Nombre", col]].rename(columns={col: "estudiantes_publicos"})
    unidades = unidades_escolares_publicas[["key", col]].rename(columns={col: "unidades_escolares_publicas"})

    datos = pd.merge(alumnos, unidades, on="key", how="inner")

    # Añadir cluster y (opcional) nombre “canon” desde cluster
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"])

    datos["estudiantes_publicos"] = pd.to_numeric(datos["estudiantes_publicos"], errors="coerce")
    datos["unidades_escolares_publicas"] = pd.to_numeric(datos["unidades_escolares_publicas"], errors="coerce")

    datos["unidades_escolares_publicas_por_100_alumnos"] = (
        (datos["unidades_escolares_publicas"] / datos["estudiantes_publicos"]) * 100
    ).replace([float("inf"), -float("inf")], 0).fillna(0)

    # más unidades por 100 alumnos => mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["unidades_escolares_publicas_por_100_alumnos"].mean()
        desviacion = grupo["unidades_escolares_publicas_por_100_alumnos"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)

        return grupo

    # Con/sin cluster
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "unidades_escolares_publicas_por_100_alumnos"}
    )

    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}unidades_esc_pub_100_alumnos_pub.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 3) PROCESAR AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2022, 2023, 2024]
Archivo del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/
Archivo del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\774014307.py:133: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\774014307.py:133: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_at

### **unidadEsc/100_alumnos_priv**

In [67]:
estudiantes_privados = pd.read_csv("data/educacion/estudiantes_privados.csv", sep=";")
unidades_escolares_privadas = pd.read_csv("data/educacion/unidades_esc_privados.csv", sep=";")

In [68]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 0) PREPARAR DATAFRAMES (asumo que ya existen)
#    - estudiantes_privados
#    - unidades_escolares_privadas
#    - municipios_cluster
# ============================================================

# --- Estudiantes privados ---
estudiantes_privados = estudiantes_privados.copy()
estudiantes_privados["Nombre"] = estudiantes_privados["Nombre"].apply(normalize_nombre)
estudiantes_privados["key"] = estudiantes_privados["Nombre"].apply(canon_key)
year_cols_est = [c for c in estudiantes_privados.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_est:
    estudiantes_privados[c] = estudiantes_privados[c].apply(parse_es_number)

# --- Unidades escolares privadas ---
unidades_escolares_privadas = unidades_escolares_privadas.copy()
unidades_escolares_privadas["Nombre"] = unidades_escolares_privadas["Nombre"].apply(normalize_nombre)
unidades_escolares_privadas["key"] = unidades_escolares_privadas["Nombre"].apply(canon_key)
year_cols_uni = [c for c in unidades_escolares_privadas.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_uni:
    unidades_escolares_privadas[c] = unidades_escolares_privadas[c].apply(parse_es_number)

# --- Clusters ---
municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)

municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 1) AÑOS A PROCESAR: comunes y >= 2022
# ============================================================
years_est = {int(c) for c in year_cols_est}
years_uni = {int(c) for c in year_cols_uni}

years_to_process = sorted(y for y in (years_est.intersection(years_uni)) if y >= 2022)
print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 2) FUNCIÓN POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    alumnos = estudiantes_privados[["key", "Nombre", col]].rename(columns={col: "estudiantes_privados"})
    unidades = unidades_escolares_privadas[["key", col]].rename(columns={col: "unidades_escolares_privadas"})

    # INNER como tu versión (solo municipios que estén en ambos)
    datos = pd.merge(alumnos, unidades, on="key", how="inner")

    # Añadir cluster y nombre final desde cluster si existe
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"], errors="ignore")

    # Numéricos
    datos["estudiantes_privados"] = pd.to_numeric(datos["estudiantes_privados"], errors="coerce")
    datos["unidades_escolares_privadas"] = pd.to_numeric(datos["unidades_escolares_privadas"], errors="coerce")

    datos["unidades_escolares_priv_100_alumnos_priv"] = (
        (datos["unidades_escolares_privadas"] / datos["estudiantes_privados"]) * 100
    )
    datos["unidades_escolares_priv_100_alumnos_priv"] = (
        datos["unidades_escolares_priv_100_alumnos_priv"]
        .replace([np.inf, -np.inf], np.nan)
        .fillna(0)
    )

    # más es mejor
    datos["type"] = 1

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["unidades_escolares_priv_100_alumnos_priv"].mean()
        desviacion = grupo["unidades_escolares_priv_100_alumnos_priv"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)
        return grupo

    # Con/sin cluster (por si algún municipio no tiene cluster)
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "unidades_escolares_priv_100_alumnos_priv"}
    )

    # Guardar
    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}unidades_esc_priv_100_alumnos_priv.csv", index=False)
    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 3) PROCESAR AÑOS
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2022, 2023, 2024]
Archivo del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/
Archivo del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\325690880.py:142: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\325690880.py:142: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_at

### **bibliotecas/100_alumnos**

In [69]:
# Leer datos
estudiantes_publicos = pd.read_csv(r"data/educacion/estudiantes_publicos.csv", sep=";")
estudiantes_privados = pd.read_csv(r"data/educacion/estudiantes_privados.csv", sep=";")
bibliotecas_publicas = pd.read_csv(r"data/educacion/bibliotecas_total.csv", sep=";")

In [70]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

# -----------------------------
# Helpers
# -----------------------------
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"
    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# ============================================================
# 0) PREPARAR DATAFRAMES (asumo que ya existen)
#    - estudiantes_publicos
#    - estudiantes_privados
#    - bibliotecas_publicas
#    - municipios_cluster
# ============================================================

def preparar_df_ancho(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    df = df.copy()
    df["Nombre"] = df["Nombre"].apply(normalize_nombre)
    df["key"] = df["Nombre"].apply(canon_key)
    year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
    for c in year_cols:
        df[c] = df[c].apply(parse_es_number)
    return df, year_cols

estudiantes_publicos, year_cols_pub = preparar_df_ancho(estudiantes_publicos)
estudiantes_privados, year_cols_priv = preparar_df_ancho(estudiantes_privados)
bibliotecas_publicas, year_cols_bib = preparar_df_ancho(bibliotecas_publicas)

municipios_cluster = municipios_cluster.copy()
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)
municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# ============================================================
# 1) AÑOS A PROCESAR: comunes (pub, priv, bib) y >= 2022
# ============================================================
years_pub = {int(c) for c in year_cols_pub}
years_priv = {int(c) for c in year_cols_priv}
years_bib = {int(c) for c in year_cols_bib}

years_to_process = sorted(
    y for y in (years_pub.intersection(years_priv).intersection(years_bib))
    if y >= 2022
)

print("Años a procesar (>=2022):", years_to_process)

# ============================================================
# 2) FUNCIÓN POR AÑO
# ============================================================
def procesar_anio(anio: int):
    col = str(int(anio))

    # Alumnos públicos + privados
    alumnos_pub = estudiantes_publicos[["key", "Nombre", col]].rename(columns={col: "estudiantes_publicos"})
    alumnos_priv = estudiantes_privados[["key", col]].rename(columns={col: "estudiantes_privados"})

    alumnos_total = pd.merge(alumnos_pub, alumnos_priv, on="key", how="inner")

    alumnos_total["estudiantes_publicos"] = pd.to_numeric(alumnos_total["estudiantes_publicos"], errors="coerce").fillna(0)
    alumnos_total["estudiantes_privados"] = pd.to_numeric(alumnos_total["estudiantes_privados"], errors="coerce").fillna(0)

    alumnos_total["estudiantes_total"] = alumnos_total["estudiantes_publicos"] + alumnos_total["estudiantes_privados"]

    # Bibliotecas públicas
    bib = bibliotecas_publicas[["key", col]].rename(columns={col: "bibliotecas_publicas"})

    # Merge final
    datos = pd.merge(alumnos_total[["key", "Nombre", "estudiantes_total"]], bib, on="key", how="inner")

    datos["estudiantes_total"] = pd.to_numeric(datos["estudiantes_total"], errors="coerce")
    datos["bibliotecas_publicas"] = pd.to_numeric(datos["bibliotecas_publicas"], errors="coerce")

    datos["bibliotecas_100_alumnos"] = (datos["bibliotecas_publicas"] / datos["estudiantes_total"]) * 100
    datos["bibliotecas_100_alumnos"] = datos["bibliotecas_100_alumnos"].replace([np.inf, -np.inf], np.nan).fillna(0)

    datos["type"] = 1

    # Añadir cluster + nombre final desde cluster si existe
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )
    datos["Nombre"] = datos["Nombre_cluster"].fillna(datos["Nombre"])
    datos = datos.drop(columns=["Nombre_cluster"], errors="ignore")

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo["bibliotecas_100_alumnos"].mean()
        desviacion = grupo["bibliotecas_100_alumnos"] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)
        return grupo

    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Salida
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": "bibliotecas_100_alumnos"}
    )

    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}bibliotecas_100_alumnos.csv", index=False)

    print(f"Archivo del {anio} exportado a {output_path}")

# ============================================================
# 3) PROCESAR
# ============================================================
for y in years_to_process:
    procesar_anio(y)


Años a procesar (>=2022): [2022, 2023, 2024]
Archivo del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/
Archivo del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/
Archivo del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\1275778568.py:145: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\1275778568.py:145: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_

### **idiomas/10_alumnos & extraescolares/10_alumnos**

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

# Cargar los archivos CSV
extraescolares = pd.read_csv(r"data/educacion/actividades_extraescolares_institucionales.csv", sep=';', encoding='ISO-8859-1')
opciones = pd.read_csv(r"data/educacion/opciones_linguisticas.csv", sep=';', encoding='ISO-8859-1')
centros = pd.read_csv(r"data/educacion/centros_educativos.csv", sep=';', encoding='ISO-8859-1')

# Clave de unión
clave = "centro_codigo"

# Columnas necesarias de centros
columnas_centros = [clave, "municipio_nombre", "municipio_codigo"]
centros_info = centros[columnas_centros]

# --- EXTRAESCOLARES ---

# Contar número de actividades por centro
extraescolares_actividades = (
    extraescolares
    .groupby(clave)
    .agg(n_actividades_extraescolares=('actividad_extraexcolar_descrip', 'nunique'))
    .reset_index()
)

# Añadir municipio a cada centro
extraescolares_municipio = extraescolares_actividades.merge(centros_info, on=clave, how="left")

# Agrupar por municipio y sacar media
media_extraescolares_por_municipio = (
    extraescolares_municipio
    .groupby(['municipio_codigo', 'municipio_nombre'])
    .agg(media_actividades_extraescolares=('n_actividades_extraescolares', 'mean'))
    .reset_index()
)

out_dir = os.path.dirname(r"external_data/educacion/media_actividades_extraescolares_por_municipio.csv")
if out_dir:  # solo si no es ''
    os.makedirs(out_dir, exist_ok=True)

# Guardar CSV de actividades extraescolares
media_extraescolares_por_municipio.to_csv("external_data/educacion/media_actividades_extraescolares_por_municipio.csv", index=False, sep=';', encoding='utf-8-sig')

# --- OPCIONES LINGÜÍSTICAS ---

# Crear una columna para contar número de idiomas por centro
idiomas_cols = ["british", "seccion_ling_ingles", "seccion_ling_frances", "seccion_ling_aleman"]
opciones["n_idiomas"] = opciones[idiomas_cols].notna().sum(axis=1)

# Sumar por centro
opciones_idiomas = (
    opciones
    .groupby(clave)
    .agg(n_opciones_idiomas=('n_idiomas', 'sum'))
    .reset_index()
)

# Añadir municipio
opciones_municipio = opciones_idiomas.merge(centros_info, on=clave, how="left")

# Agrupar por municipio y sacar media
media_idiomas_por_municipio = (
    opciones_municipio
    .groupby(['municipio_codigo', 'municipio_nombre'])
    .agg(media_opciones_idiomas=('n_opciones_idiomas', 'mean'))
    .reset_index()
)

# Crear carpeta si aplica
out_dir = os.path.dirname(r"external_data/educacion/media_opciones_linguisticas_por_municipio.csv")
if out_dir:  # solo si no es ''
    os.makedirs(out_dir, exist_ok=True)

# Guardar CSV de opciones lingüísticas
media_idiomas_por_municipio.to_csv("external_data/educacion/media_opciones_linguisticas_por_municipio.csv", index=False, sep=';', encoding='utf-8-sig')


In [72]:
import os
import re
import unicodedata
import numpy as np
import pandas as pd

# =========================
# PATHS
# =========================
MUNICIPIOS_CLUSTER_CSV = r"external_data/municipios_con_cluster.csv"   # sep=","
ESTUDIANTES_PUBLICOS_CSV = r"data/educacion/estudiantes_publicos.csv" # sep=";"
LAU_CSV = r"external_data/lau_id_nombre.csv"                          # sep=","

indicadores = {
    "extraescolares": r"external_data/educacion/media_actividades_extraescolares_por_municipio.csv",
    "idiomas": r"external_data/educacion/media_opciones_linguisticas_por_municipio.csv"
}

# =========================
# Helpers
# =========================
def parse_es_number(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s.lower() == "nan":
        return None
    s = s.replace(".", "").replace(",", ".")
    try:
        return float(s)
    except ValueError:
        return None

def normalize_nombre(nombre):
    """Normaliza nombres tipo '28004 Álamo, El' -> 'Álamo (El)'."""
    if pd.isna(nombre):
        return nombre
    s = str(nombre).strip()
    s = re.sub(r"^\d+\s+", "", s)  # quita INE si viene

    # 'Álamo, El' -> 'Álamo (El)'
    if ", " in s and "(" not in s and ")" not in s:
        parts = s.split(", ")
        if len(parts) == 2 and parts[1] in {"La", "El", "Los", "Las"}:
            s = f"{parts[0].strip()} ({parts[1].strip()})"

    s = re.sub(r"\s+", " ", s)
    return s

def canon_key(nombre):
    """Clave canónica para casar aunque haya tildes, comas, guiones, etc."""
    s = normalize_nombre(nombre)
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))  # quita tildes
    s = s.lower()
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

# =========================
# Cargar datos base
# =========================
municipios_cluster = pd.read_csv(MUNICIPIOS_CLUSTER_CSV, sep=",", dtype=str)
estudiantes_publicos = pd.read_csv(ESTUDIANTES_PUBLICOS_CSV, sep=";", dtype=str)
lau = pd.read_csv(LAU_CSV, sep=",", dtype=str)

# --- Preparar LAU (lista completa de municipios) ---
lau_unique = lau.drop_duplicates("id").rename(columns={"nombre": "municipio_nombre"}).copy()
lau_unique["municipio_nombre"] = lau_unique["municipio_nombre"].apply(normalize_nombre)
lau_unique["key"] = lau_unique["municipio_nombre"].apply(canon_key)

# Si hay duplicados de key por nombres raros, nos quedamos con uno
lau_unique = lau_unique.drop_duplicates("key").reset_index(drop=True)

# --- Preparar estudiantes (ancho por años) ---
estudiantes_publicos["Nombre"] = estudiantes_publicos["Nombre"].apply(normalize_nombre)
estudiantes_publicos["key"] = estudiantes_publicos["Nombre"].apply(canon_key)

year_cols_est = [c for c in estudiantes_publicos.columns if re.fullmatch(r"\d{4}", str(c))]
for c in year_cols_est:
    estudiantes_publicos[c] = estudiantes_publicos[c].apply(parse_es_number)

# Años a procesar (>=2022)
years_to_process = sorted([int(y) for y in year_cols_est if int(y) >= 2022])
print("Años a procesar (>=2022):", years_to_process)

# --- Preparar clusters ---
municipios_cluster["Nombre"] = municipios_cluster["Nombre"].apply(normalize_nombre)
municipios_cluster["key"] = municipios_cluster["Nombre"].apply(canon_key)
municipios_cluster_ren = municipios_cluster.rename(columns={"Cluster": "cluster"})
municipios_cluster_ren["cluster"] = pd.to_numeric(municipios_cluster_ren["cluster"], errors="coerce")

# =========================
# Función principal
# =========================
def procesar_anio_indicador_cluster(anio: int, indicador: str, ruta_csv: str):
    print(f"\nProcesando {indicador} - {anio}")

    # Cargar medias del indicador
    df_ind = pd.read_csv(ruta_csv, sep=";", dtype=str)

    # Detectar columna media (media_*)
    media_cols = [col for col in df_ind.columns if str(col).startswith("media_")]
    if not media_cols:
        raise ValueError(f"No se encontró ninguna columna que empiece por 'media_' en {ruta_csv}")
    col_media = media_cols[0]

    # Normalizar nombre y key en indicador
    df_ind["municipio_nombre"] = df_ind["municipio_nombre"].apply(normalize_nombre)
    df_ind["key"] = df_ind["municipio_nombre"].apply(canon_key)
    df_ind[col_media] = df_ind[col_media].apply(parse_es_number)

    # Rellenar municipios faltantes (según LAU) con media=0
    base = lau_unique[["key", "municipio_nombre"]].copy()
    datos_ind = pd.merge(base, df_ind[["key", col_media]], on="key", how="left")
    datos_ind[col_media] = pd.to_numeric(datos_ind[col_media], errors="coerce").fillna(0.0)

    # Cargar alumnos (públicos) del año
    col = str(int(anio))
    alumnos = estudiantes_publicos[["key", "Nombre", col]].rename(columns={col: "estudiantes_total"}).copy()
    alumnos["estudiantes_total"] = pd.to_numeric(alumnos["estudiantes_total"], errors="coerce").fillna(0)

    # Merge indicador + alumnos (por key)
    datos = pd.merge(datos_ind, alumnos[["key", "estudiantes_total"]], on="key", how="left")
    datos["estudiantes_total"] = pd.to_numeric(datos["estudiantes_total"], errors="coerce").fillna(0)

    # Ratio por 10 alumnos (evitar inf)
    ratio_col = f"{indicador}_10_alumnos"
    datos[ratio_col] = (datos[col_media] / datos["estudiantes_total"]) * 10
    datos[ratio_col] = datos[ratio_col].replace([np.inf, -np.inf], np.nan).fillna(0)

    # type = 1 (más es mejor)
    datos["type"] = 1

    # Añadir cluster (por key)
    datos = pd.merge(
        datos,
        municipios_cluster_ren[["key", "Nombre", "cluster"]],
        on="key",
        how="left",
        suffixes=("", "_cluster")
    )

    # Nombre final: preferimos el del cluster si existe, si no el de LAU
    datos["Nombre"] = datos["Nombre"].fillna(datos["municipio_nombre"])
    datos = datos.drop(columns=["municipio_nombre"], errors="ignore")

    def calcular_atractividad_por_grupo(grupo: pd.DataFrame) -> pd.DataFrame:
        grupo = grupo.copy()
        media = grupo[ratio_col].mean()
        desviacion = grupo[ratio_col] - media
        max_dev = desviacion.abs().max()

        grupo["atractividad"] = (desviacion / max_dev) if (max_dev and max_dev != 0) else 0

        grupo["atractividad_0_100"] = grupo.apply(
            lambda row: 100 * ((row["atractividad"] + 1) / 2) if row["type"] == 1
            else 100 - (100 * ((row["atractividad"] + 1) / 2)),
            axis=1
        ).round(2)
        return grupo

    # Con/sin cluster para evitar NaN en groupby
    con_cluster = datos[datos["cluster"].notna()].copy()
    sin_cluster = datos[datos["cluster"].isna()].copy()

    con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)

    if not sin_cluster.empty:
        sin_cluster["atractividad_0_100"] = 0.0

    datos_final = pd.concat([con_cluster, sin_cluster], ignore_index=True)

    # Resultado final
    resultado = datos_final[["Nombre", "atractividad_0_100"]].rename(
        columns={"atractividad_0_100": ratio_col}
    )

    # Guardar
    output_path = f"data_interfaz/educacion/normalizacion_final/{anio}/"
    os.makedirs(output_path, exist_ok=True)
    resultado.to_csv(f"{output_path}{indicador}_10_alumnos.csv", index=False)
    print(f"{indicador} del {anio} exportado a {output_path}")

# =========================
# Ejecutar para años >= 2022 e indicadores
# =========================
for indicador, ruta in indicadores.items():
    for anio in years_to_process:
        procesar_anio_indicador_cluster(anio, indicador, ruta)


Años a procesar (>=2022):

C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\643531639.py:168: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)


 [2022, 2023, 2024]

Procesando extraescolares - 2022
extraescolares del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/

Procesando extraescolares - 2023
extraescolares del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/

Procesando extraescolares - 2024
extraescolares del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/

Procesando idiomas - 2022
idiomas del 2022 exportado a data_interfaz/educacion/normalizacion_final/2022/

Procesando idiomas - 2023
idiomas del 2023 exportado a data_interfaz/educacion/normalizacion_final/2023/

Procesando idiomas - 2024
idiomas del 2024 exportado a data_interfaz/educacion/normalizacion_final/2024/


C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\643531639.py:168: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_atractividad_por_grupo)
C:\Users\aitan\AppData\Local\Temp\ipykernel_19988\643531639.py:168: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  con_cluster = con_cluster.groupby("cluster", group_keys=False).apply(calcular_at